Датасет отсюда https://www.kaggle.com/deepcontractor/aircraft-accidents-failures-hijacks-dataset

In [23]:
import numpy as np
import pandas as pd
import dill

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import LogisticRegression
from datetime import datetime
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [2]:
df = pd.read_csv('incident/Aircraft_Incident_Dataset.csv').fillna('')

In [3]:
import re

def str_date_to_float(d):
    #print(f'date before {d}')
    d.strip()
    #### Исправляем кривые даты. Заменяем ?? на число или месяц
    d_fxd = re.sub('^[?]*-', '01-', d, 1)
    d_fxd = re.sub('-[?]*-', '-JAN-', d_fxd)
    #print(f'date after {d_fxd}')
    #### Иногда приходит месяц как число, которое > 12. Меняю его на 12 и в зависимости от типа даты применяю 
    # шаблон для конвертации
    if(d_fxd[3:5].isnumeric()):
        if (int(d_fxd[3:5]) > 12):
            d_fxd = d_fxd[0:3] + '12' + d_fxd[5:]
        dt =  datetime.strptime(d_fxd, '%d-%m-%Y')
    else:
        dt = datetime.strptime(d_fxd, '%d-%b-%Y') 
    ####    
    epoch = datetime.utcfromtimestamp(0)
    total_seconds =  (dt - epoch).total_seconds()
    # total_seconds - милисекунды
    return total_seconds

In [4]:
df['Incident_Date'] = df['Incident_Date'].apply(lambda x : str_date_to_float(x))

date before 03-JAN-2022
date after 03-JAN-2022
date before 04-JAN-2022
date after 04-JAN-2022
date before 05-JAN-2022
date after 05-JAN-2022
date before 08-JAN-2022
date after 08-JAN-2022
date before 12-JAN-2022
date after 12-JAN-2022
date before 22-JAN-2022
date after 22-JAN-2022
date before 31-JAN-2022
date after 31-JAN-2022
date before 05-FEB-2022
date after 05-FEB-2022
date before 08-FEB-2022
date after 08-FEB-2022
date before 11-FEB-2022
date after 11-FEB-2022
date before 14-FEB-2022
date after 14-FEB-2022
date before 15-FEB-2022
date after 15-FEB-2022
date before 02-JAN-2021
date after 02-JAN-2021
date before 02-JAN-2021
date after 02-JAN-2021
date before 09-JAN-2021
date after 09-JAN-2021
date before 09-JAN-2021
date after 09-JAN-2021
date before 12-JAN-2021
date after 12-JAN-2021
date before 19-JAN-2021
date after 19-JAN-2021
date before 19-JAN-2021
date after 19-JAN-2021
date before 23-JAN-2021
date after 23-JAN-2021
date before 23-JAN-2021
date after 23-JAN-2021
date before 2

date after 10-JUL-2018
date before 11-JUL-2018
date after 11-JUL-2018
date before 16-JUL-2018
date after 16-JUL-2018
date before 16-JUL-2018
date after 16-JUL-2018
date before 16-JUL-2018
date after 16-JUL-2018
date before 20-JUL-2018
date after 20-JUL-2018
date before 21-JUL-2018
date after 21-JUL-2018
date before 22-JUL-2018
date after 22-JUL-2018
date before 23-JUL-2018
date after 23-JUL-2018
date before 27-JUL-2018
date after 27-JUL-2018
date before 27-JUL-2018
date after 27-JUL-2018
date before 28-JUL-2018
date after 28-JUL-2018
date before 28-JUL-2018
date after 28-JUL-2018
date before 28-JUL-2018
date after 28-JUL-2018
date before 30-JUL-2018
date after 30-JUL-2018
date before 31-JUL-2018
date after 31-JUL-2018
date before 01-AUG-2018
date after 01-AUG-2018
date before 04-AUG-2018
date after 04-AUG-2018
date before 07-AUG-2018
date after 07-AUG-2018
date before 07-AUG-2018
date after 07-AUG-2018
date before 09-AUG-2018
date after 09-AUG-2018
date before 09-AUG-2018
date after 09

date before 29-JUN-2014
date after 29-JUN-2014
date before 02-JUL-2014
date after 02-JUL-2014
date before 04-JUL-2014
date after 04-JUL-2014
date before 06-JUL-2014
date after 06-JUL-2014
date before 06-JUL-2014
date after 06-JUL-2014
date before 07-JUL-2014
date after 07-JUL-2014
date before 10-JUL-2014
date after 10-JUL-2014
date before 14-JUL-2014
date after 14-JUL-2014
date before 14-JUL-2014
date after 14-JUL-2014
date before 15-JUL-2014
date after 15-JUL-2014
date before 16-JUL-2014
date after 16-JUL-2014
date before 16-JUL-2014
date after 16-JUL-2014
date before 17-JUL-2014
date after 17-JUL-2014
date before 17-JUL-2014
date after 17-JUL-2014
date before 20-JUL-2014
date after 20-JUL-2014
date before 20-JUL-2014
date after 20-JUL-2014
date before 20-JUL-2014
date after 20-JUL-2014
date before 22-JUL-2014
date after 22-JUL-2014
date before 22-JUL-2014
date after 22-JUL-2014
date before 23-JUL-2014
date after 23-JUL-2014
date before 24-JUL-2014
date after 24-JUL-2014
date before 2

date before 24-AUG-2010
date after 24-AUG-2010
date before 25-AUG-2010
date after 25-AUG-2010
date before 25-AUG-2010
date after 25-AUG-2010
date before 25-AUG-2010
date after 25-AUG-2010
date before 25-AUG-2010
date after 25-AUG-2010
date before 26-AUG-2010
date after 26-AUG-2010
date before 31-AUG-2010
date after 31-AUG-2010
date before 03-SEP-2010
date after 03-SEP-2010
date before 03-SEP-2010
date after 03-SEP-2010
date before 05-SEP-2010
date after 05-SEP-2010
date before 07-SEP-2010
date after 07-SEP-2010
date before 07-SEP-2010
date after 07-SEP-2010
date before 13-SEP-2010
date after 13-SEP-2010
date before 15-SEP-2010
date after 15-SEP-2010
date before 22-SEP-2010
date after 22-SEP-2010
date before 22-SEP-2010
date after 22-SEP-2010
date before 24-SEP-2010
date after 24-SEP-2010
date before 01-OCT-2010
date after 01-OCT-2010
date before 02-OCT-2010
date after 02-OCT-2010
date before 03-OCT-2010
date after 03-OCT-2010
date before 06-OCT-2010
date after 06-OCT-2010
date before 1

date before 01-SEP-2006
date after 01-SEP-2006
date before 02-SEP-2006
date after 02-SEP-2006
date before 02-SEP-2006
date after 02-SEP-2006
date before 07-SEP-2006
date after 07-SEP-2006
date before 08-SEP-2006
date after 08-SEP-2006
date before 08-SEP-2006
date after 08-SEP-2006
date before 15-SEP-2006
date after 15-SEP-2006
date before 15-SEP-2006
date after 15-SEP-2006
date before 17-SEP-2006
date after 17-SEP-2006
date before 26-SEP-2006
date after 26-SEP-2006
date before 26-SEP-2006
date after 26-SEP-2006
date before 29-SEP-2006
date after 29-SEP-2006
date before 29-SEP-2006
date after 29-SEP-2006
date before 02-OCT-2006
date after 02-OCT-2006
date before 02-OCT-2006
date after 02-OCT-2006
date before 03-OCT-2006
date after 03-OCT-2006
date before 03-OCT-2006
date after 03-OCT-2006
date before 03-OCT-2006
date after 03-OCT-2006
date before 10-OCT-2006
date after 10-OCT-2006
date before 10-OCT-2006
date after 10-OCT-2006
date before 12-OCT-2006
date after 12-OCT-2006
date before 1

...

date after 18-MAR-1944
date before 18-MAR-1944
date after 18-MAR-1944
date before 20-MAR-1944
date after 20-MAR-1944
date before 21-MAR-1944
date after 21-MAR-1944
date before 21-MAR-1944
date after 21-MAR-1944
date before 21-MAR-1944
date after 21-MAR-1944
date before 21-MAR-1944
date after 21-MAR-1944
date before 22-MAR-1944
date after 22-MAR-1944
date before 22-MAR-1944
date after 22-MAR-1944
date before 22-MAR-1944
date after 22-MAR-1944
date before 22-MAR-1944
date after 22-MAR-1944
date before 24-MAR-1944
date after 24-MAR-1944
date before 24-MAR-1944
date after 24-MAR-1944
date before 24-MAR-1944
date after 24-MAR-1944
date before 24-MAR-1944
date after 24-MAR-1944
date before 24-MAR-1944
date after 24-MAR-1944
date before 24-MAR-1944
date after 24-MAR-1944
date before 25-MAR-1944
date after 25-MAR-1944
date before 25-MAR-1944
date after 25-MAR-1944
date before 25-MAR-1944
date after 25-MAR-1944
date before 25-MAR-1944
date after 25-MAR-1944
date before 25-MAR-1944
date after 25

date after 22-SEP-1944
date before 22-SEP-1944
date after 22-SEP-1944
date before 22-SEP-1944
date after 22-SEP-1944
date before 23-SEP-1944
date after 23-SEP-1944
date before 23-SEP-1944
date after 23-SEP-1944
date before 23-SEP-1944
date after 23-SEP-1944
date before 23-SEP-1944
date after 23-SEP-1944
date before 23-SEP-1944
date after 23-SEP-1944
date before 23-SEP-1944
date after 23-SEP-1944
date before 23-SEP-1944
date after 23-SEP-1944
date before 23-SEP-1944
date after 23-SEP-1944
date before 23-SEP-1944
date after 23-SEP-1944
date before 23-SEP-1944
date after 23-SEP-1944
date before 23-SEP-1944
date after 23-SEP-1944
date before 23-SEP-1944
date after 23-SEP-1944
date before 23-SEP-1944
date after 23-SEP-1944
date before 23-SEP-1944
date after 23-SEP-1944
date before 24-SEP-1944
date after 24-SEP-1944
date before 24-SEP-1944
date after 24-SEP-1944
date before 24-SEP-1944
date after 24-SEP-1944
date before 24-SEP-1944
date after 24-SEP-1944
date before 24-SEP-1944
date after 24

date after 09-JUL-1943
date before 09-JUL-1943
date after 09-JUL-1943
date before 09-JUL-1943
date after 09-JUL-1943
date before 09-JUL-1943
date after 09-JUL-1943
date before 10-JUL-1943
date after 10-JUL-1943
date before 10-JUL-1943
date after 10-JUL-1943
date before 10-JUL-1943
date after 10-JUL-1943
date before 10-JUL-1943
date after 10-JUL-1943
date before 10-JUL-1943
date after 10-JUL-1943
date before 10-JUL-1943
date after 10-JUL-1943
date before 11-JUL-1943
date after 11-JUL-1943
date before 11-JUL-1943
date after 11-JUL-1943
date before 11-JUL-1943
date after 11-JUL-1943
date before 11-JUL-1943
date after 11-JUL-1943
date before 11-JUL-1943
date after 11-JUL-1943
date before 11-JUL-1943
date after 11-JUL-1943
date before 11-JUL-1943
date after 11-JUL-1943
date before 11-JUL-1943
date after 11-JUL-1943
date before 11-JUL-1943
date after 11-JUL-1943
date before 11-JUL-1943
date after 11-JUL-1943
date before 11-JUL-1943
date after 11-JUL-1943
date before 11-JUL-1943
date after 11

date before 03-OCT-1942
date after 03-OCT-1942
date before 07-OCT-1942
date after 07-OCT-1942
date before 09-OCT-1942
date after 09-OCT-1942
date before 10-OCT-1942
date after 10-OCT-1942
date before 11-OCT-1942
date after 11-OCT-1942
date before 15-OCT-1942
date after 15-OCT-1942
date before 16-OCT-1942
date after 16-OCT-1942
date before 19-OCT-1942
date after 19-OCT-1942
date before 20-OCT-1942
date after 20-OCT-1942
date before 22-OCT-1942
date after 22-OCT-1942
date before 22-OCT-1942
date after 22-OCT-1942
date before 22-OCT-1942
date after 22-OCT-1942
date before 23-OCT-1942
date after 23-OCT-1942
date before 23-OCT-1942
date after 23-OCT-1942
date before 25-OCT-1942
date after 25-OCT-1942
date before 26-OCT-1942
date after 26-OCT-1942
date before 30-OCT-1942
date after 30-OCT-1942
date before 31-OCT-1942
date after 31-OCT-1942
date before ??-OCT-1942
date after 01-OCT-1942
date before 01-NOV-1942
date after 01-NOV-1942
date before 02-NOV-1942
date after 02-NOV-1942
date before 0

date after 02-DEC-1938
date before 03-DEC-1938
date after 03-DEC-1938
date before 06-DEC-1938
date after 06-DEC-1938
date before 08-DEC-1938
date after 08-DEC-1938
date before 09-DEC-1938
date after 09-DEC-1938
date before 10-DEC-1938
date after 10-DEC-1938
date before 11-DEC-1938
date after 11-DEC-1938
date before 03-JAN-1937
date after 03-JAN-1937
date before 20-JAN-1937
date after 20-JAN-1937
date before 26-JAN-1937
date after 26-JAN-1937
date before 09-FEB-1937
date after 09-FEB-1937
date before 16-FEB-1937
date after 16-FEB-1937
date before 18-FEB-1937
date after 18-FEB-1937
date before 13-MAR-1937
date after 13-MAR-1937
date before 16-MAR-1937
date after 16-MAR-1937
date before 24-MAR-1937
date after 24-MAR-1937
date before 25-MAR-1937
date after 25-MAR-1937
date before ??-MAR-1937
date after 01-MAR-1937
date before ??-MAR-1937
date after 01-MAR-1937
date before 01-APR-1937
date after 01-APR-1937
date before 03-APR-1937
date after 03-APR-1937
date before 06-APR-1937
date after 06

In [5]:
# Парсим колонку Onboard_Total на три
onboard_columns = {'fatalities': [], 'occupants': [], 'survived': []}
for onboard_total in df['Onboard_Total']:
    onboard_total = onboard_total.split('/')
    fatalities = onboard_total[0].split(':')[1].replace(' ', '')
    occupants = onboard_total[1].split(':')[1].replace(' ', '')
    if fatalities != '' and occupants != '':
        fatalities, occupants = int(fatalities), int(occupants)
        survived = int(occupants - fatalities)
        onboard_columns['fatalities'].append(fatalities)
        onboard_columns['occupants'].append(occupants)
        onboard_columns['survived'].append(survived)
    else:
        onboard_columns['fatalities'].append(0)
        onboard_columns['occupants'].append(0)
        onboard_columns['survived'].append(0)
df['fatalities_total'] = onboard_columns['fatalities']
df['occupants_tottal'] = onboard_columns['occupants']
df['survived_total'] = onboard_columns['survived']

In [6]:
# Создаем таргет. Основываемся на данных из колонки Fatalities (0 -> 0; else -> 1)
df['target'] = np.where(df['Fatalities'] == 0, 0, 1)
df

,Incident_Date,Aircaft_Model,Aircaft_Registration,Aircaft_Operator,Aircaft_Nature,Incident_Category,Incident_Cause(es),Incident_Location,Aircaft_Damage_Type,Date,...,Aircaft_First_Flight,Aircraft_Phase,Departure_Airport,Destination_Airport,Ground_Casualties,Collision_Casualties,fatalities_total,occupants_tottal,survived_total,target
0,1.641168e+09,British Aerospace 4121 Jetstream 41,ZS-NRJ,SA Airlink,Domestic Non Scheduled Passenger,Accident | repairable-damage,"Airplane - Engines, Airplane - Engines - Prop/...",near Venetia Mine...,Substantial,Monday 3 January 2022,...,1995-05-19 (26 years 8 months),Landing (LDG),Johannesburg-O.R. Tambo International Airport ...,"Venetia Mine Airport (FAVM) , South Africa",,,0,7,7,0
1,1.641254e+09,British Aerospace 3101 Jetstream 31,HR-AYY,LANHSA,Domestic Scheduled Passenger,Accident | repairable-damage,"Airplane - Undercarriage, Airplane - Undercarr...",Roatán-Juan ...,Substantial,Tuesday 4 January 2022,...,1985,Landing (LDG),La Ceiba-Goloson International Airport (LCE/MH...,Roatán-Juan Manuel Gálvez International Airpor...,,,0,19,19,0
2,1.641341e+09,Boeing 737-4H6,EP-CAP,Caspian Airlines,Domestic Scheduled Passenger,Accident | repairable-damage,"Airplane - Undercarriage, Airplane - Undercarr...",Isfahan-Shah...,Substantial,Wednesday 5 January 2022,...,1992-09-18 (29 years 4 months),Landing (LDG),"Mashhad Airport (MHD/OIMM) , Iran","Isfahan-Shahid Beheshti Airport (IFN/OIFM) , Iran",,,0,116,116,0
3,1.641600e+09,Tupolev Tu-204-100C,RA-64032,"Cainiao, opb Aviastar-TU",Cargo,Accident | hull-loss,"Cargo - Fire/smoke, Result - Damaged on the gr...",Hangzhou-Xia...,Destroyed,Saturday 8 January 2022,...,2002-07-18 (19 years 6 months),Standing (STD),Hangzhou-Xiaoshan International Airport (HGH/Z...,"Novosibirsk-Tolmachevo Airport (OVB/UNNT) , Ru...",,,0,8,8,0
4,1.641946e+09,Beechcraft 200 Super King Air,,private,Illegal Flight,"Criminal occurrence (sabotage, shoot down) | h...",Result - Damaged on the ground,"Machakilha, ...",Damaged beyond repair,Wednesday 12 January 2022,...,,Standing (STD),?,?,,,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23514,-1.573171e+09,Handley Page O/400,G-EAMC,Handley Page Transport,Unknown,Accident | hull-loss,"Result - Emergency, forced landing - Outside a...",near El Shereik,Damaged beyond repair,Wednesday 25 February 1920,...,,Unknown (UNK),"Aswan Airport (ASW/HESN) , Egypt","Khartoum-Civil Airport (KRT/HSSS) , Sudan",,,0,4,4,0
23515,-1.562285e+09,Handley Page O/400,G-EAKE,Handley Page Transport,Demonstration,Accident | hull-loss,"Result - Emergency, forced landing - Outside a...",Östanå,Damaged beyond repair,Wednesday 30 June 1920,...,1919,En route (ENR),"Stockholm (unknown airport) , Sweden","Kjeller Air Base (ENKJ) , Norway",,,0,2,2,0
23516,-1.547856e+09,Handley Page O/400,G-EAMA,Handley Page Transport,International Scheduled Passenger,Accident | hull-loss,Result - CFIT - Level ground,Golders Green,Damaged beyond repair,Tuesday 14 December 1920,...,,Initial climb (ICL),"London-Cricklewood Aerodrome , United Kingdom","Paris-Le Bourget Airport (LBG/LFPB) , France",,,4,8,4,1
23517,-1.591056e+09,Caproni Ca.48,,Caproni,Passenger,Accident | hull-loss,Info-Unavailable,Verona,Destroyed,Saturday 2 August 1919,...,1919,En route (ENR),"Venice-Marco Polo Airport (VCE/LIPZ) , Italy","Milano-Taliedo Airport , Italy",,,14,14,0,1


In [7]:
categorical_columns = ['Aircaft_Model', 'Aircaft_Registration', 'Aircaft_Operator', 'Aircaft_Nature','Incident_Category', 'Incident_Cause(es)','Aircaft_Damage_Type', 'Aircaft_Engines', 'Aircraft_Phase', 'Incident_Location', 'Date', 'Time', 'Arit', 'Onboard_Crew', 'Onboard_Passengers', 'Onboard_Total', 'Fatalities', 'Aircaft_First_Flight', 'Departure_Airport', 'Destination_Airport', 'Ground_Casualties', 'Collision_Casualties']
continuous_columns = ['Incident_Date', 'fatalities_total', 'occupants_tottal', 'survived_total']

In [8]:
X=df.drop('target', axis=1)
y=df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=0)

X_test.to_csv('model/X_test.csv', index=None)
y_test.to_csv('model/y_test.csv', index=None)
X_train.to_csv('model/X_train.csv', index=None)
y_train.to_csv('model/y_train.csv', index=None)

In [9]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]


class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        x_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in x_columns:
                 X[col_] = 0
        return X[self.columns]


In [ ]:
df['Incident_Location']

In [10]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col))
            ])
    final_transformers.append((cont_col, cont_transformer))

In [11]:
feats = FeatureUnion(final_transformers)

In [40]:
results_tab = {
    'Approach' : [],
    'Precision' : [],
    'Recall' : [],
    'F-Score' : [],
    'Roc-Auc' : []
}

In [13]:
pipeline_lr = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])

In [ ]:
df.columns

In [14]:
# Кросс-валидация
cv_scores = cross_val_score(pipeline_lr, X_train, y_train, cv=3, scoring='roc_auc')
cv_score = np.mean(cv_scores)

In [15]:
#обучим наш пайплайн
pipeline_lr.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Aircaft_Model',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Aircaft_Model')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Aircaft_Model'))])),
                                                ('Aircaft_Registration',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Aircaft_Registration')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Aircaft_Registration'))])),
                                                ('Aircaft_Op...
                              

In [18]:
# Сохраняем модель
with open('model/model_lr.dill', 'wb') as output:
        dill.dump(pipeline_lr, output)

In [19]:
# Загружаем модель
with open('model/model_lr.dill', 'rb') as input:
        pipeline_lr = dill.load(input) 

In [31]:
#наши прогнозы для тестовой выборки
preds = pipeline_lr.predict_proba(X_test)[:, 1]
preds[:10]

array([0.56370287, 0.4491308 , 0.38162452, 0.48783846, 0.41751585,
       0.50182926, 0.47210268, 0.54408134, 0.54044897, 0.55357656])

In [32]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

In [33]:
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)

print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.598568, F-Score=nan, Precision=0.000, Recall=0.000


<ipython-input-33-27f25929018e>:1: RuntimeWarning: invalid value encountered in true_divide
  fscore = (2 * precision * recall) / (precision + recall)


In [34]:
roc_auc = roc_auc_score(y_true=y_test, y_score=pipeline_lr.predict_proba(X_test)[:,1])
print(f"Roc auc: {roc_auc}")

Roc auc: 0.536668254968567


In [37]:
cnf_matrix = confusion_matrix(y_test, preds>thresholds[ix])
    
TN = cnf_matrix[0][0]
FN = cnf_matrix[1][0]
TP = cnf_matrix[1][1]
FP = cnf_matrix[0][1]
print(cnf_matrix)

[[731   0]
 [445   0]]


In [42]:
results_tab['Approach'].append('Linear Regression')
results_tab['Precision'].append(precision[ix])
results_tab['Recall'].append(recall[ix])
results_tab['F-Score'].append(fscore[ix])
results_tab['Roc-Auc'].append(roc_auc)

In [43]:
pd.DataFrame(data=results_tab)

,Approach,Precision,Recall,F-Score,Roc-Auc
0,Linear Regression,0.0,0.0,NaN,0.536668
